## Importing Libaries

In [1]:
import random
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import math
import random
import pandas as pd

## Creating Model & Prediction functions

In [3]:
def create_model(state_size, action_size):
    model = Sequential()
    model.add(Dense(units=64, input_dim=state_size, activation="relu"))
    model.add(Dense(units=32, activation="relu"))
    model.add(Dense(units=8, activation="relu"))
    model.add(Dense(action_size, activation="linear"))
    model.compile(loss="mse", optimizer=Adam(learning_rate=0.001))
    return model

def act(model, state, epsilon):
    if random.random() <= epsilon:
        return random.randrange(action_size)
    options = model.predict(state)
    return np.argmax(options[0])

## Helper Functions

In [4]:
def formatPrice(n):
    return ("-Rs." if n < 0 else "Rs.") + "{0:.2f}".format(abs(n))

def getStockDataVec(data):
    vec = []
    for index, row in data.iterrows():
        low, high, open, close, date = row
        vec.append(close)
    return vec

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def getState(data, t, n):
    d = t - n + 1
    block = (
        data[d : t + 1] if d >= 0 else -d * [data[0]] + data[0 : t + 1]
    )
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))
    return np.array([res])

## Loading Data

In [6]:
data = pd.read_csv("HistoricalData_1715123520180.csv")
df_reverse_both = data.iloc[::-1, ::-1]

## Splitting Data into Test, Train

In [7]:
X = list(df_reverse_both["Close/Last"])
X = [float(x) for x in X]

validation_size = 0.2
train_size = int(len(X) * (1-validation_size))

X_train, X_test = X[0:train_size], X[train_size:len(X)]
data = X_train
l = len(data) - 1
batch_size = 32

## Hyperparameters

In [8]:
state_size = 1
action_size = 3
episode_count = 1

model = create_model(state_size, action_size)
gamma = 0.95
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
batch_size = 32

## Running Agent Model

In [16]:
inventory = []
for e in range(episode_count + 1):
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getState(data, 0, state_size + 1)
    total_profit = 0
    for t in range(l):
        action = act(model, state, epsilon)
        # hold
        next_state = getState(data, t + 1, state_size + 1)
        reward = 0
        # buy
        if action == 1:
            inventory.append(data[t])
            print("Buy: " + formatPrice(data[t]))
        # sell
        elif action == 2 and len(inventory) > 0:
            bought_price = window_size_price = inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print(
                "Sell: "
                + formatPrice(data[t])
                + " | Profit: "
                + formatPrice(data[t] - bought_price)
            )
        done = True if t == l - 1 else False
        state = next_state
        if done:
            print("-------------    -------------------")
            print("Total Profit: " + formatPrice(total_profit))
            print("--------------------------------")


Episode 0/1
Buy: Rs.2811.87
Sell: Rs.2834.41 | Profit: Rs.22.54
Buy: Rs.2850.96
Sell: Rs.2876.32 | Profit: Rs.25.36
Buy: Rs.2840.23
Sell: Rs.2856.27 | Profit: Rs.16.04
Buy: Rs.2826.06
Sell: Rs.2802.39 | Profit: -Rs.23.67
Buy: Rs.2783.02
Sell: Rs.2788.86 | Profit: Rs.5.84
Buy: Rs.2752.06
Sell: Rs.2744.45 | Profit: -Rs.7.61
Buy: Rs.2879.84
Buy: Rs.2889.67
Sell: Rs.2917.75 | Profit: Rs.37.91
Sell: Rs.2926.46 | Profit: Rs.36.79
Buy: Rs.2917.38
Buy: Rs.2913.78
Sell: Rs.2924.92 | Profit: Rs.7.54
Buy: Rs.2941.76
Sell: Rs.2964.33 | Profit: Rs.50.55
Buy: Rs.2973.01
Buy: Rs.2995.82
Sell: Rs.2990.41 | Profit: Rs.48.65
Sell: Rs.2975.95 | Profit: Rs.2.94
Buy: Rs.2979.63
Buy: Rs.2993.07
Buy: Rs.3013.77
Buy: Rs.3014.30
Sell: Rs.2984.42 | Profit: -Rs.11.40
Buy: Rs.2995.11
Buy: Rs.2985.03
Buy: Rs.3005.47
Buy: Rs.3003.67
Sell: Rs.3025.86 | Profit: Rs.46.23
Buy: Rs.3020.97
Buy: Rs.3013.18
Sell: Rs.2953.56 | Profit: -Rs.39.51
Sell: Rs.2844.74 | Profit: -Rs.169.03
Buy: Rs.2938.09
Sell: Rs.2918.65 | Profit: